In [8]:
import sys
sys.path.append('../functions')

from app import *
from config import *

In [11]:
# Quick inspection of json loading for config.py
import json

from reportlab.lib.units import cm
from decouple import config

headers = {
    'Access-Control-Allow-Origin': '*',
    'Access-Control-Allow-Credentials': True,
    'Content-Type': 'application/json'
}

# ------------------------------ #
# Begin parameter change block   #
# ------------------------------ #
CRON_MINS = 5
production_env = True

fixScores = None
# ----------------- #
# End               #
# ----------------- #

general_prefix = "/" if production_env else ""
api_prefix = ".."

education_quickscan_info = {
    "class_name":"Education",
    "class_attrs":{
        "score_offset":[261, 261, 261],
        "offset_vertical":[556, 402, 258],
        "offset_horizontal":[75, 75, 75],
        "variables_in_order":['autonomie', 'relatie', 'competentie'],
        "api_prefix":api_prefix,
        "general_prefix":general_prefix,
        "report_folder":"onderwijs_quickscan",
        "out_filename":"Rapport Quickscan.pdf",
        "template_link": "https://preview.mailerlite.com/o3u9q3"
    }
}

vitality_info = {
    "class_name":"Vitality",
    "class_attrs":{
        "score_offset":[261],
        "offset_vertical":[310],
        "offset_horizontal":[127],
        "variables_in_order":['result'],
        "api_prefix":api_prefix,
        "general_prefix":general_prefix,
        "report_folder":"stress_quickscan",
        "out_filename":"Rapport Quickscan.pdf",
        "template_link": "https://preview.mailerlite.com/i6e0h6"
    }
}

init_info = [
    {
        "type": 'Education Quickscan',
        "class_info": education_quickscan_info,
        "stage": [
            {
                "stage": "dev",
                "form_id": "v3q6i6Lm"
            }
        ]
    },
    {
        "type": 'Vitality',
        "class_info": vitality_info,
        "stage": [
            {
                "stage": "dev",
                "form_id": "vaXpvkUJ"
            },
            {
                "stage": "prod",
                "form_id": "vOPz3zmu"
            }
        ]
    },
    {
        "type": 'Vitality Ichtus',
        "class_info": json.load(open("report_templates/vitality_ichtus/config.json")),
        "stage": [
            {
                "stage": "dev",
                "form_id": "QBycyvIp"
            },
            {
                "stage": "prod",
                "form_id": "ynK8MSXN"
            }
        ]
    }
]

In [9]:
def selfscan_cron(event, context):
    try:
        emails = event['emails']
    except:
        print("Emails not passed as call parameters")

    reports_sent = []

    for form_id in form_ids:
        request = json.dumps({'form_info': form_id})

        #########################################################
        # Only fetch data younger than x minutes                #
        #########################################################
        form_data = extract_form_data(request, None)
        form_data = json.loads(form_data['body'])
        response_data = form_data['response_data']
        response_data_cp = response_data.copy()

        print("Length of original reponse items: " + str(len(response_data['items'])))

        for i, response in enumerate(response_data['items'][:]):
            try:  # If we parsed a list with email to send emails to
                print(emails)
                email = [e for e in response['answers'] if e['type'] == 'email'][0]['email']

                if email not in emails:
                    print("Email not found")
                    response_data_cp['items'].remove(response)
                else:
                    print("Email found")

            except NameError:  # If we use no list and simply want to use the CRON job
                print("Length of reponse items: " + str(len(response_data['items'])))
                submitted_time = datetime.fromisoformat(response['submitted_at'].replace('Z', '+00:00'))
                curr_time = datetime.now(timezone.utc)

                minutes_diff = (curr_time - submitted_time).total_seconds() / 30.0
                if minutes_diff <= 80:
                    print(minutes_diff)
                else:
                    response_data_cp['items'].remove(response)

        print("Length of new response items: " + str(len(response_data_cp['items'])))
        if len(response_data_cp['items']) == 0:
            reports_sent.append(0)
            break

        form_data['response_data'] = response_data_cp
        form_data = {
            'body': json.dumps(form_data)
        }

        df = construct_df_from_form_data(form_data, None)

        scores = generate_scores(df, None)

        # Insert class info
        z = scores['body']
        z = json.loads(z)

        z['class_info'] = form_id['class_info']
        scores['body'] = json.dumps(z)
        # End

        pdfs = generate_pdfs(scores, None)

        reports = send_reports(pdfs, None)
        reports = reports['body']
        reports = json.loads(reports)

        reports_sent.append(reports['reports_sent'])

    result = {
        'forms': form_ids,
        'reports_sent': reports_sent
    }

    return {
        'statusCode': 200,
        'headers': headers,
        'body': json.dumps(result)
    }

In [10]:
selfscan_cron(None, None)

Emails not passed as call parameters
Length of original reponse items: 7
Length of reponse items: 7
18.260225233333333
Length of reponse items: 7
37.5268919
Length of reponse items: 7
Length of reponse items: 6
Length of reponse items: 5
Length of reponse items: 4
Length of reponse items: 3
Length of new response items: 2
Number of fields in survey: 55
Number of respondents in survey: 2
['x99gKiLqaXNu', '3yjrODxMzkye', 'rUItwipczhf6', 'VDfTl5sMaH58', 'tJ6CsLVYCihb', 'CVPcteWhGdGx', 'B38mnxYzS6j4', 'YwoFjv2tDa30', '0l6slP4NVqpZ', 'Tq8bFbeTse23', '2OYCZTJUtkum', '1b5N7mQC4g0z', 'aGgrl8WMMDCT', '3PBwBcfyb0Ok', 'M3MjbLa4bUGM', 'h615YIohDy5b', 'yZABWYFPOAEV', 'PTWIDQnT0SMC', 'seAIDqJw4qCn', 'l6gqx8NN2012', 'wZp2fs3gM1BA', 'lCWbI4IX2Icz', 'j0mFbIFGq3E7', 'bmjcFTFKq5PA', 'GFRipnKSES0T', 'bA4eHY1jJ2Dz', '8hXTzcqbpHCR', 'KReXquemAXZs', '8H7vJMtGrOUc', 'WFFAoDTRezk7', 'adxADJ8JLV0w', 'DegfKXPHCU00', 'QrWthb0zCGNQ', 'OfKhsR4koyQO', 'V7kip99kRTgb', 'eI0FGBHbiFAi', 'pMDdNl5Rg9pf', 'qQcDWmFshdVq', '

{'statusCode': 200,
 'headers': {'Access-Control-Allow-Origin': '*',
  'Access-Control-Allow-Credentials': True,
  'Content-Type': 'application/json'},
 'body': '{"forms": [{"form_id": "QBycyvIp", "class_info": {"class_name": "Vitality Ichtus", "class_attrs": {"page_attrs": {"Werkdruk": {"text": ["Je geeft aan dat je altijd of meestal je werk af krijgt, en dat je nooit of niet vaak je werk minder goed doet dan je eigenlijk zou willen, om het werk af te krijgen. Dit wijst op een lage werkdruk.", "Je geeft aan dat het je soms niet lukt om je werk af te krijgen, en dat je soms je werk minder goed doet dan je eigenlijk zou willen, om het werk af te krijgen. Dit wijst op een enigszins hoge werkdruk.", "Je geeft aan dat je je werk vaak of nooit afkrijgt, en dat je vaak of altijd je werk minder goed doet dan je eigenlijk zou willen, om het werk af te krijgen. Dit wijst op een hoge werkdruk."], "offset_vertical": 627, "offset_horizontal": -125, "score_offset": 6.05, "cutoff_score": [1, 2, 3], 

In [ ]:
vitalityIchtus_info

In [ ]:
init_info

In [ ]:
form_ids = []

form_ids.append({
    'form_id':fetch_form_id('Vitality Ichtus', 'dev', init_info),
    'class_info':vitalityIchtus_info
})

form_id = form_ids[0]
print(form_id)

In [ ]:
event = json.dumps({'form_info':form_id})
def extract_form_data(event, context):
    data = json.loads(event)
    form_info = data['form_info']

    params = {'page_size': 1000}
    responses = requests.get(TF_BASE_URL + "/forms/" + form_info['form_id'] + "/responses", headers=auth, params=params)
    form = requests.get(TF_BASE_URL + "/forms/" + form_info['form_id'], headers=auth)

    resp = json.loads(responses.content)
    frm = json.loads(form.content)

    print(frm)

    result = {
        "form_info":form_info,
        "form_data":frm,
        "response_data":resp
    }

    return {
        'statusCode': 200,
        'headers': headers,
        'body': json.dumps(result)
    }

form_data = extract_form_data(event,None)

In [ ]:
# data = event['body']
data = form_data['body']
data = json.loads(data)

frm = data['form_data']
resp = data['response_data']
frm_info = data['form_info']

form = Form(frm['title'])

cols = []
col_ids = []
rows = []

opinion_fields = [field for field in frm['fields'] if field['type'] == 'opinion_scale']
total_score = sum((field['properties']['steps']-1) if field['properties']['start_at_one'] else field['properties']['steps'] for field in opinion_fields)

for col in frm['fields']:
    if col['type'] != "statement":
        col_ids.append(col['id'])
        if col['type'] == "email":
            cols.append("email")
        else:
            cols.append(col['title'])

print("Number of fields in survey:", str(len(frm['fields'])))
print("Number of respondents in survey:", str(len(resp['items'])))

print(col_ids)

for item in resp['items']:
    row = []

    i = j = 0
    while i <= len(item['answers'])-1:
        answer = item['answers'][i]

        if answer['field']['id'] == col_ids[j]:
            tpe = answer['type']
            row.append(answer[tpe])
            i += 1
            j += 1
        else:
            row.append('no answer')
            j += 1

    rows.append(row)

df = pd.DataFrame(rows, columns=cols)
json_df = df.to_dict(orient="dict")

dd = defaultdict(list)

# variables_list = np.array([item['variables'] for item in resp['items']])

# Use vars when declared in the typeform, use calculated score otherwise
if frm_info['class_info']['class_name'] != "Vitality HLR" and frm_info['class_info']['class_name'] != "Vitality Ichtus":
    variables_list = np.array([item['variables'] for item in resp['items']])
    print("Used vars")

    score_list = []
    # For every item (respondent) in list of key: value pairs
    for item in variables_list:
        temp_dict = {}
        # For every key in the item (e.g. 'anatomie')
        for i in item:
            # Make a new key in a temporary dict where the value is equal to the total score by the respondent
            temp_dict[i['key']] = i['number']

        # Append temporary dictionary of respondent to main dict containing all respondents
        score_list.append(temp_dict)

    # New code
    frm_vars = frm['variables']
    max_scores = frm_vars.copy()
    min_scores = frm_vars.copy()

    del min_scores['score']

    for logic in frm['logic']:
        max_val = 0

        target_dict = {}
        for actions in logic['actions']:
            action, target, source = actions.values()

            tgt_var = target['target']['value']
            tgt_val = target['value']['value']

            if action == 'add' and tgt_val > frm_vars[tgt_var]:
                target_dict[tgt_var] = tgt_val

        max_scores = dict(Counter(max_scores) + Counter(target_dict))
    # End
else:
    ## New part
    # If a question dictionary exists we want to calculate the scores based on this file
    if 'question_dict' in frm_info['class_info']['class_attrs']:
        f = open("report_templates/"+frm_info['class_info']['class_attrs']['question_dict'])
        question_dict = json.load(f)

    fields = []

    for field in frm['fields']:
        item = dict()

        item['id'] = field['id']
        item['title'] = field['title']
        item['type'] = field['type']
        if 'validations' in field:
            item['required'] = field['validations']['required']
        else:
            item['required'] = False

        fields.append(item)

    field_list = []
    for q in question_dict:
        category = q['category']

        for item in q['items']:
            item_dict = dict()
            item_dict['category'] = category

            for field in fields:
                if field['title'] == item['question']:
                    item_dict['id'] = field['id']
                    item_dict['question'] = item['question']
                    item_dict['type'] = field['type']
                    item_dict['required'] = field['required']

            field_list.append(item_dict)

    logic_list = []
    for logic in frm['logic']:
        for action in logic['actions']:
            try:
                action_info = {'ref': action['condition']['vars'][1]['value'], 'value': action['details']['value']['value']}

                logic_list.append(action_info)
            except:
                continue

    answer_list = []
    url_list = []
    print("Number of respondents:", str(len(resp['items'])))
    for item in resp['items']:
        # Code block for when a URL is used in stead of email
        for v in item['variables']:
            if v['key'] == 'url':
                url_list.append(v['number'])
        # end

        resp_list = []
        for answer in item['answers']:
            if answer['type'] == "number":
                answer_info = {'field_id': answer['field']['id'], 'number': answer['number']}
            elif answer['type'] == "choice":
                answer_info = {'field_id': answer['field']['id'], 'ref': answer['choice']['ref']}

            resp_list.append(answer_info)

        answer_list.append(resp_list)

    # variables list must return [{'key':'Werkdruk', 'number':'_score'},{'key':'Emotionele belasting', 'number':'_score'}] for every respondent
    cats_score = []
    cats_unique = [q['category'] for q in question_dict]

    variables_list = []
    for i in range(len(resp['items'])): # In the range of the respondents
        variables_list_resp = []

        for cat in cats_unique:
            cat_score = {"key": cat, "number":[]}
            variables_list_resp.append(cat_score)

        for field in field_list: # For every question
            if field['type'] == 'multiple_choice' and field['required']:
                answer = [answer for answer in answer_list[i] if answer['field_id'] == field['id']][0]
                logic = [logic for logic in logic_list if logic['ref'] == answer['ref']][0]['value']
                for cat in variables_list_resp:
                    if field['category'] == cat['key']:
                        cat['number'].append(logic)
                        break

            else:
                for cat in variables_list_resp:
                    if field['category'] == cat['key'] and field['required']:
                        answer = [answer for answer in answer_list[i] if answer['field_id'] == field['id']][0]
                        cat['number'].append(answer['number'])
                        break

        for var in variables_list_resp:
            var['number'] = np.mean(var['number'])

        variables_list.append(variables_list_resp)

    score_list = []
    # For every item (respondent) in list of key: value pairs
    for item in variables_list:
        temp_dict = {}
        # For every key in the item
        for i in item:
            # Make a new key in a temporary dict where the value is equal to the total score by the respondent
            temp_dict[i['key']] = i['number']

        # Append temporary dictionary of respondent to main dict containing all respondents
        score_list.append(temp_dict)

# Default code to transform into defaultdict
dd = create_dict_with_lists(score_list)

response = {
    'scores':dd,
    'urls':url_list,
    'dataframe':json_df,
    'form_info':frm_info,
    'form':json.dumps(form.__dict__)
}

rtrn = {
    'statusCode': 200,
    'headers': headers,
    'body': json.dumps(response)
}

print(rtrn)

In [ ]:
print("Starting score generation")

data = rtrn['body']
data = json.loads(data)
#########################################################
# Construct basic dataset                               #
#########################################################

df = pd.DataFrame()

try:
    df['email'] = data['dataframe']['email'].values()
except KeyError:
    df['email'] = None

#########################################################
# Create scores                                         #
#########################################################
for key, value in data['scores'].items():
    df[key] = data['scores'][key]

df["url"] = data['urls']

#     df['raw_scores'] = data['scores']
#     df['final_scores'] = (df['raw_score']/data['total_scores'])*MAX_SCORE
#     df['final_scores'] = np.round(df['final_scores']).astype("int")

#     df = df[['email','final_scores']]

#########################################################
# DF to dict                                            #
#########################################################
json_df = df.to_dict(orient="records")

#########################################################
# Create final dict                                     #
#########################################################
response = {
    'variables':list(data['scores'].keys()),
    'dataframe':json_df,
    'form_info':data['form_info']
}

rtrn2 = {
    'statusCode': 200,
    'headers': headers,
    'body': json.dumps(response)
}

print(rtrn2)

In [ ]:
general_prefix = "" # Local

vitalityIchtus_info = {
    "class_name":"Vitality Ichtus",
    "class_attrs":{
        "page_attrs":{
            "Werkdruk":{
                "text":["Je geeft aan dat je altijd of meestal je werk af krijgt, en dat je nooit of niet vaak je werk minder goed doet dan je eigenlijk zou willen, om het werk af te krijgen. Dit wijst op een lage werkdruk.", "Je geeft aan dat het je soms niet lukt om je werk af te krijgen, en dat je soms je werk minder goed doet dan je eigenlijk zou willen, om het werk af te krijgen. Dit wijst op een enigszins hoge werkdruk.", "Je geeft aan dat je je werk vaak of nooit afkrijgt, en dat je vaak of altijd je werk minder goed doet dan je eigenlijk zou willen, om het werk af te krijgen. Dit wijst op een hoge werkdruk."],
                "offset_vertical":627,
                "offset_horizontal":-125,
                "score_offset":6.05,
                "cutoff_score":[1,2,3],
                "colors":['green', 'yellow', 'red'],
                "min_score":1,
                "max_score":4,
            },
            "Overuren":{
                "text":["Dit is het aantal overuren dat je gemiddeld per week maakt. Gemiddeld geven Nederlanders aan dat zij 3 overuren per week maken."],
                "offset_vertical":14.35,
                "offset_horizontal":-125,
                "score_offset":6.05,
                "cutoff_score":[0],
                "colors":['grey'],
                "min_score":0,
                "max_score":10,
            },
            "Emotionele belasting":{
                "text":["Jouw score geeft aan dat je minder emotionele belasting ervaart dan een gemiddelde werknemer.", "Je ervaart de emotionele belasting als hoger dan gemiddeld. Dat betekent dat je te maken hebt met hoge emotionele taakeisen."],
                "offset_vertical":204,
                "offset_horizontal":-125,
                "score_offset":6.05,
                "cutoff_score":[0,1.68],
                "colors":['yellow', 'red'],
                "min_score":1,
                "max_score":4,
            },
            "Cognitieve belasting":{
                "text":["Jouw score geeft aan dat je minder mentale belasting ervaart in het werk dan een gemiddelde werknemer.", "Jouw score geeft aan dat je de cognitieve belasting in het werk als hoger ervaart dan een gemiddelde werknemer."],
                "offset_vertical":22.00,
                "offset_horizontal":-125,
                "score_offset":6.05,
                "cutoff_score":[0,2.99],
                "colors":['yellow', 'red'],
                "min_score":1,
                "max_score":4,
            },
            "Sociale steun leidinggevende":{
                "text":["Je score geeft aan dat je weinig sociale steun ervaart van jouw leidinggevende.", "Je score geeft aan dat je beperkt sociale steun ervaart van jouw leidinggevende.", "Jouw score geeft aan dat je evenveel sociale steun ervaart van je leidinggevende als de gemiddelde werknemer.", "Jouw score geeft aan dat je meer sociale steun ervaart van je leidinggevende dan de gemiddelde werknemer."],
                "offset_vertical":14.24,
                "offset_horizontal":-125,
                "score_offset":6.05,
                "cutoff_score":[1,2,2.5,3],
                "colors":['red', 'orange', 'yellow', 'green'],
                "min_score":1,
                "max_score":4,
            },
            "Functionele steun leidinggevende":{
                "text":["Je geeft aan dat je weinig functionele steun van je leidinggevende ervaart.","Je geeft aan dat je beperkt functionele steun van je leidinggevende ervaart.","Je geeft aan dat je evenveel functionele steun van je leidinggevende ervaart als de gemiddelde werknemer.","Je geeft aan dat je meer functionele steun van je leidinggevende ervaart dan de gemiddelde werknemer."],
                "offset_vertical":6.81,
                "offset_horizontal":-125,
                "score_offset":6.05,
                "cutoff_score":[1,2,2.5,3],
                "colors":['red', 'orange', 'yellow', 'green'],
                "min_score":1,
                "max_score":4,
            },
            "Sociale steun collegae":{
                "text":["Je score geeft aan dat je weinig sociale steun ervaart van je collega's.","Je score geeft aan dat je beperkt sociale steun ervaart van je collega's.","Je score geeft aan dat je evenveel sociale steun ervaart van je collega's als de gemiddelde werknemer.","Je score geeft aan dat je meer sociale steun ervaart van collega's dan de gemiddelde werknemer."],
                "offset_vertical":21.66,
                "offset_horizontal":-125,
                "score_offset":6.05,
                "cutoff_score":[1,2,3,4],
                "colors":['red', 'orange', 'yellow', 'green'],
                "min_score":1,
                "max_score":4,
            },
            "Functionele steun collegae":{
                "text":["Je score geeft aan dat je weinig functionele steun ervaart van je collega's.","Je score geeft aan dat je beperkt functionele steun ervaart van je collega's.","Je score geeft aan dat je evenveel funtionele steun ervaart van je collega's als de gemiddelde werknemer.","Je score geeft aan dat je meer functionele steun ervaart van collega's dan de gemiddelde werknemer."],
                "offset_vertical":14.65,
                "offset_horizontal":-125,
                "score_offset":6.05,
                "cutoff_score":[1,2,3,4],
                "colors":['red', 'orange', 'yellow', 'green'],
                "min_score":1,
                "max_score":4,
            },
            "Autonomie":{
                "text":["Jouw score geeft aan dat je minder invloed op het werk hebt dan de gemiddelde werknemer. Als je bij jezelf merkt dat je behoefte hebt aan meer invloed op het werk is het wellicht verstandig daar eens met jouw leidinggevende over te praten.","Jouw score geeft aan dat je meer invloed op het werk hebt dan de gemiddelde werknemer."],
                "offset_vertical":7.91,
                "offset_horizontal":-125,
                "score_offset":6.05,
                "cutoff_score":[0,2.35],
                "colors":['yellow', 'green'],
                "min_score":1,
                "max_score":4,
            },
            "Ongewenst gedrag intern":{
                "text":["Je geeft aan nooit last te hebben van ongewenst gedrag door internen (collega's en leidinggevende).","Je geeft aan soms last te hebben van ongewenst gedrag door internen (collega's of leidinggevende).","Je geeft aan vaak last te hebben van ongewenst gedrag door internen (collega's of leidinggevende)."],
                "offset_vertical":21.78,
                "offset_horizontal":-132,
                "score_offset":6.05,
                "cutoff_score":[0,1.01,2.51],
                "colors":['green','yellow','red'],
                "min_score":1,
                "max_score":4,
            },
            "Ongewenst gedrag extern":{
                "text":["Je geeft aan helemaal geen last te hebben van ongewenst gedrag door externen (leerlingen, ouders e.d.).","Je geeft aan soms te maken te hebben met ongewenst gedrag door externen (leerlingen, ouders e.d.).","Je geeft aan vaak last te hebben van ongewenst gedrag door externen (leerlingen, ouders e.d.)."],
                "offset_vertical":14.79,
                "offset_horizontal":-132,
                "score_offset":6.05,
                "cutoff_score":[0,1.01,2.51],
                "colors":['green','yellow','red'],
                "min_score":1,
                "max_score":4,
            },
            "Ontwikkelmogelijkheden":{
                "text":["Je geeft aan dat jouw leidinggevende de ontwikkeling van je kennis en vaardigheden niet stimuleert.","Je geeft aan dat jouw leidinggevende de ontwikkeling van je kennis en vaardigheden in beperkte mate stimuleert.","Je geeft aan dat jouw leidinggevende de ontwikkeling van je kennis en vaardigheden in grote mate stimuleert."],
                "offset_vertical":7.73,
                "offset_horizontal":-132,
                "score_offset":6.05,
                "cutoff_score":[0.5,1.5,2.5],
                "colors":['red','yellow','green'],
                "min_score":0.5,
                "max_score":3.5,
            },
            "Burn-out klachten":{
                "text":["Je hebt minder burn-out klachten dan de gemiddelde medewerker.","Je hebt meer burn-out klachten dan de gemiddelde medewerker.","Je hebt veel vaker last van burn-out klachten dan de gemiddelde medewerker."],
                "offset_vertical":22.10,
                "offset_horizontal":-132,
                "score_offset":6.05,
                "cutoff_score":[1,2.03,3.21],
                "colors":['green','yellow','red'],
                "min_score":1,
                "max_score":7,
            },
            "Bevlogenheid":{
                "text":["Jouw score op bevlogenheid is lager dan die van de gemiddelde medewerker.","Jouw score op bevlogenheid is hoger dan de gemiddelde medewerker."],
                "offset_vertical":14.89,
                "offset_horizontal":-132,
                "score_offset":6.05,
                "cutoff_score":[0,5.53],
                "colors":['yellow','green'],
                "min_score":1,
                "max_score":7,
            },
            "Tevredenheid":{
                "text":["Je bent minder tevreden met jouw werk en arbeidsomstandigheden dan de gemiddelde medewerker.","Je bent meer tevreden met jouw werk en arbeidsomstandigheden dan de gemiddelde medewerker."],
                "offset_vertical":7.84,
                "offset_horizontal":-132,
                "score_offset":6.05,
                "cutoff_score":[0,3.79],
                "colors":['yellow', 'green'],
                "min_score":1,
                "max_score":5,
            },
            "Organisatiecultuur":{
                "text":["In jouw organisatie heerst een open cultuur waar ruimte is om duurzame inzetbaarheid te bespreken.","In jouw organisatie heerst niet altijd een open cultuur waar ruimte is om duurzame inzetbaarheid te bespreken.","In jouw organisatie heerst geen open cultuur en er lijkt geen ruimte om duurzame inzetbaarheid te bespreken."],
                "offset_vertical":21.24,
                "offset_horizontal":-132,
                "score_offset":6.05,
                "cutoff_score":[0,2.51,4],
                "colors":['green', 'yellow', 'red'],
                "min_score":1,
                "max_score":5,
            },
        },
        "api_prefix":api_prefix,
        "general_prefix":general_prefix,
        "report_folder":"vitality_ichtus",
        "out_filename":"Rapport Vitaliteit.pdf",
        "template_link": "https://preview.mailerlite.com/q9y8u7b3r8",
        "question_dict": "vitality_ichtus/questions.json"
    }
}

In [ ]:
class VitalityIchtus(PDF):
    def __init__(self,
                 attrs,
                 person = None):

        self.attrs = attrs

        for k, v in self.attrs.items():
            setattr(self, k, v)

    def init_super(self):
        PDF.__init__(self,
                     self.attrs,
                     person = self.person
                     )

    def cutoff(self, subject):
        value = getattr(self.person, subject)
        subject = self.page_attrs[subject]

        length = len(subject['text'])
        for i in range(length):
            if i < length-1:
                if subject['cutoff_score'][i] <= value < subject['cutoff_score'][i + 1]:
                    cutoff_color = subject['colors'][i]
                    cutoff_text = subject['text'][i]

            elif i == length-1:
                if value >= subject['cutoff_score'][i]:
                    cutoff_color = subject['colors'][i]
                    cutoff_text = subject['text'][i]

        return cutoff_color, cutoff_text


    def create_canvas(self):
        font_folder = str(Path("", "fonts"))
        pdfmetrics.registerFont(TTFont('Montserrat-SemiBold', Path(font_folder,'Montserrat-SemiBold.ttf')))
        pdfmetrics.registerFont(TTFont('Montserrat-Regular', Path(font_folder,'Montserrat-Regular.ttf')))
        pdfmetrics.registerFont(TTFont('Montserrat-Medium', Path(font_folder,'Montserrat-Medium.ttf')))

        report_filename = "PSA rapport vitaliteit V.2"
        pages = range(10)

        height = 19

        c, existing_pdf = self.prep_pdf(report_filename, pages)
        c.showPage()
        c.showPage()

        ## Werkdruk
        subject = "Werkdruk"
        sub_attrs = self.page_attrs[subject]
        print(percentage_from_score(sub_attrs['min_score'],sub_attrs['max_score'],getattr(self.person, subject)))
        self.draw_score(c,
                        sub_attrs['score_offset']*cm,
                        sub_attrs['offset_horizontal'],
                        percentage_from_score(sub_attrs['min_score'],sub_attrs['max_score'],getattr(self.person, subject)),
                        sub_attrs['offset_vertical'],
                        height = height)

        c.drawImage("resources/images/arrow_"+self.cutoff(subject)[0]+".png",
                    (-10.93*cm)+(0.4*cm),
                    18.7*cm,
                    height=25,
                    preserveAspectRatio=True)

        frame = Frame(1.5*cm, 16.1*cm,8*cm,100, showBoundary=0)
        story = [Paragraph(self.cutoff(subject)[1], custom_style)]
        frame.addFromList(story,c)

        ## Overuren
        subject = "Overuren"
        c.drawImage("resources/images/arrow_"+self.cutoff(subject)[0]+".png",
                    (-10.93*cm)+(0.4*cm),
                    11*cm,
                    height=25,
                    preserveAspectRatio=True)

        sub_attrs = self.page_attrs[subject]

        self.draw_score(c,
                        sub_attrs['score_offset']*cm,
                        sub_attrs['offset_horizontal'],
                        percentage_from_score(sub_attrs['min_score'],sub_attrs['max_score'],getattr(self.person, subject)),
                        sub_attrs['offset_vertical']*cm,
                        height = height)

        frame = Frame(1.5*cm, 8.3*cm,8*cm,100, showBoundary=0)
        story = [Paragraph(self.cutoff(subject)[1], custom_style)]
        frame.addFromList(story,c)

        ## Emotionele belasting
        subject = "Emotionele belasting"
        sub_attrs = self.page_attrs[subject]
        self.draw_score(c,
                        sub_attrs['score_offset']*cm,
                        sub_attrs['offset_horizontal'],
                        percentage_from_score(sub_attrs['min_score'],sub_attrs['max_score'],getattr(self.person, subject)),
                        sub_attrs['offset_vertical'],
                        height = height)

        c.drawImage("resources/images/arrow_"+self.cutoff(subject)[0]+".png",
                    (-10.93*cm)+(0.4*cm),
                    4.1*cm,
                    height=25,
                    preserveAspectRatio=True)

        frame = Frame(1.5*cm, 1.5*cm,8*cm,100, showBoundary=0)
        story = [Paragraph(self.cutoff(subject)[1], custom_style)]
        frame.addFromList(story,c)

        c.showPage() # Werkdruk, overuren, emotionele belasting

        ## Cognitieve belasting
        subject = "Cognitieve belasting"
        sub_attrs = self.page_attrs[subject]
        self.draw_score(c,
                        sub_attrs['score_offset']*cm,
                        sub_attrs['offset_horizontal'],
                        percentage_from_score(sub_attrs['min_score'],sub_attrs['max_score'],getattr(self.person, subject)),
                        sub_attrs['offset_vertical']*cm,
                        height = height)

        c.drawImage("resources/images/arrow_"+self.cutoff(subject)[0]+".png",
                    (-10.93*cm)+(0.4*cm),
                    18.7*cm,
                    height=25,
                    preserveAspectRatio=True)

        frame = Frame(1.5*cm, 16.1*cm,8*cm,100, showBoundary=0)
        story = [Paragraph(self.cutoff(subject)[1], custom_style)]
        frame.addFromList(story,c)

        ## Sociale steun leidinggevende
        subject = "Sociale steun leidinggevende"
        c.drawImage("resources/images/arrow_"+self.cutoff(subject)[0]+".png",
                    (-10.93*cm)+(0.4*cm),
                    11.2*cm,
                    height=25,
                    preserveAspectRatio=True)

        sub_attrs = self.page_attrs[subject]
        self.draw_score(c,
                        sub_attrs['score_offset']*cm,
                        sub_attrs['offset_horizontal'],
                        percentage_from_score(sub_attrs['min_score'],sub_attrs['max_score'],getattr(self.person, subject)),
                        sub_attrs['offset_vertical']*cm,
                        height = height)

        frame = Frame(1.5*cm, 8.5*cm,8*cm,100, showBoundary=0)
        story = [Paragraph(self.cutoff(subject)[1], custom_style)]
        frame.addFromList(story,c)

        ## Functionele steun leidinggevende
        subject = "Functionele steun leidinggevende"
        sub_attrs = self.page_attrs[subject]
        self.draw_score(c,
                        sub_attrs['score_offset']*cm,
                        sub_attrs['offset_horizontal'],
                        percentage_from_score(sub_attrs['min_score'],sub_attrs['max_score'],getattr(self.person, subject)),
                        sub_attrs['offset_vertical']*cm,
                        height = height)

        c.drawImage("resources/images/arrow_"+self.cutoff(subject)[0]+".png",
                    (-10.93*cm)+(0.4*cm),
                    3.9*cm,
                    height=25,
                    preserveAspectRatio=True)

        frame = Frame(1.5*cm, 1.3*cm,8*cm,100, showBoundary=0)
        story = [Paragraph(self.cutoff(subject)[1], custom_style)]
        frame.addFromList(story,c)

        c.showPage() # Cognitieve belasting, sociale steun leidinggevende, functionele steun leidinggevende

        ## Sociale steun collegae
        subject = "Sociale steun collegae"
        sub_attrs = self.page_attrs[subject]
        self.draw_score(c,
                        sub_attrs['score_offset']*cm,
                        sub_attrs['offset_horizontal'],
                        percentage_from_score(sub_attrs['min_score'],sub_attrs['max_score'],getattr(self.person, subject)),
                        sub_attrs['offset_vertical']*cm,
                        height = height)

        c.drawImage("resources/images/arrow_"+self.cutoff(subject)[0]+".png",
                    (-10.93*cm)+(0.4*cm),
                    18.7*cm,
                    height=25,
                    preserveAspectRatio=True)

        frame = Frame(1.5*cm, 16.1*cm,8*cm,100, showBoundary=0)
        story = [Paragraph(self.cutoff(subject)[1], custom_style)]
        frame.addFromList(story,c)

        ## Functionele steun collegae,
        subject = "Functionele steun collegae"
        c.drawImage("resources/images/arrow_"+self.cutoff(subject)[0]+".png",
                    (-10.93*cm)+(0.4*cm),
                    11.4*cm,
                    height=25,
                    preserveAspectRatio=True)

        sub_attrs = self.page_attrs[subject]
        self.draw_score(c,
                        sub_attrs['score_offset']*cm,
                        sub_attrs['offset_horizontal'],
                        percentage_from_score(sub_attrs['min_score'],sub_attrs['max_score'],getattr(self.person, subject)),
                        sub_attrs['offset_vertical']*cm,
                        height = height)

        frame = Frame(1.5*cm, 8.7*cm,8*cm,100, showBoundary=0)
        story = [Paragraph(self.cutoff(subject)[1], custom_style)]
        frame.addFromList(story,c)

        ## Autonomie
        subject = "Autonomie"
        sub_attrs = self.page_attrs[subject]
        self.draw_score(c,
                        sub_attrs['score_offset']*cm,
                        sub_attrs['offset_horizontal'],
                        percentage_from_score(sub_attrs['min_score'],sub_attrs['max_score'],getattr(self.person, subject)),
                        sub_attrs['offset_vertical']*cm,
                        height = height)

        c.drawImage("resources/images/arrow_"+self.cutoff(subject)[0]+".png",
                    (-10.93*cm)+(0.4*cm),
                    4.8*cm,
                    height=25,
                    preserveAspectRatio=True)

        frame = Frame(1.5*cm, 2.2*cm,8*cm,100, showBoundary=0)
        story = [Paragraph(self.cutoff(subject)[1], custom_style)]
        frame.addFromList(story,c)

        c.showPage() # Sociale steun collega's, functionele steun collega's, autonomie

        ## Ongewenst gedrag intern
        subject = "Ongewenst gedrag intern"
        sub_attrs = self.page_attrs[subject]
        self.draw_score(c,
                        sub_attrs['score_offset']*cm,
                        sub_attrs['offset_horizontal'],
                        percentage_from_score(sub_attrs['min_score'],sub_attrs['max_score'],getattr(self.person, subject)),
                        sub_attrs['offset_vertical']*cm,
                        height = height)

        c.drawImage("resources/images/arrow_"+self.cutoff(subject)[0]+".png",
                    (-10.93*cm)+(0.4*cm),
                    18.7*cm,
                    height=25,
                    preserveAspectRatio=True)

        frame = Frame(1.5*cm, 16.1*cm,8*cm,100, showBoundary=0)
        story = [Paragraph(self.cutoff(subject)[1], custom_style)]
        frame.addFromList(story,c)

        ## Ongewenst gedrag extern
        subject = "Ongewenst gedrag extern"
        c.drawImage("resources/images/arrow_"+self.cutoff(subject)[0]+".png",
                    (-10.93*cm)+(0.4*cm),
                    11.4*cm,
                    height=25,
                    preserveAspectRatio=True)

        sub_attrs = self.page_attrs[subject]
        self.draw_score(c,
                        sub_attrs['score_offset']*cm,
                        sub_attrs['offset_horizontal'],
                        percentage_from_score(sub_attrs['min_score'],sub_attrs['max_score'],getattr(self.person, subject)),
                        sub_attrs['offset_vertical']*cm,
                        height = height)

        frame = Frame(1.5*cm, 8.7*cm,8*cm,100, showBoundary=0)
        story = [Paragraph(self.cutoff(subject)[1], custom_style)]
        frame.addFromList(story,c)

        ## Ontwikkelmogelijkheden
        subject = "Ontwikkelmogelijkheden"
        c.drawImage("resources/images/arrow_"+self.cutoff(subject)[0]+".png",
                    (-10.93*cm)+(0.4*cm),
                    4.2*cm,
                    height=25,
                    preserveAspectRatio=True)

        sub_attrs = self.page_attrs[subject]
        self.draw_score(c,
                        sub_attrs['score_offset']*cm,
                        sub_attrs['offset_horizontal'],
                        percentage_from_score(sub_attrs['min_score'],sub_attrs['max_score'],getattr(self.person, subject)),
                        sub_attrs['offset_vertical']*cm,
                        height = height)

        frame = Frame(1.5*cm, 1.56*cm,8*cm,100, showBoundary=0)
        story = [Paragraph(self.cutoff(subject)[1], custom_style)]
        frame.addFromList(story,c)

        c.showPage() # Ongewenst gedrag intern, ongewenst gedrag extern, ontwikkelmogelijkheden

        ## Burn-out klachten
        subject = "Burn-out klachten"
        sub_attrs = self.page_attrs[subject]
        self.draw_score(c,
                        sub_attrs['score_offset']*cm,
                        sub_attrs['offset_horizontal'],
                        percentage_from_score(sub_attrs['min_score'],sub_attrs['max_score'],getattr(self.person, subject)),
                        sub_attrs['offset_vertical']*cm,
                        height = height)

        c.drawImage("resources/images/arrow_"+self.cutoff(subject)[0]+".png",
                    (-10.93*cm)+(0.4*cm),
                    18.7*cm,
                    height=25,
                    preserveAspectRatio=True)

        frame = Frame(1.5*cm, 16.1*cm,8*cm,100, showBoundary=0)
        story = [Paragraph(self.cutoff(subject)[1], custom_style)]
        frame.addFromList(story,c)

        ## Bevlogenheid
        subject = "Bevlogenheid"
        c.drawImage("resources/images/arrow_"+self.cutoff(subject)[0]+".png",
                    (-10.93*cm)+(0.4*cm),
                    11.4*cm,
                    height=25,
                    preserveAspectRatio=True)

        sub_attrs = self.page_attrs[subject]
        self.draw_score(c,
                        sub_attrs['score_offset']*cm,
                        sub_attrs['offset_horizontal'],
                        percentage_from_score(sub_attrs['min_score'],sub_attrs['max_score'],getattr(self.person, subject)),
                        sub_attrs['offset_vertical']*cm,
                        height = height)

        frame = Frame(1.5*cm, 8.7*cm,8*cm,100, showBoundary=0)
        story = [Paragraph(self.cutoff(subject)[1], custom_style)]
        frame.addFromList(story,c)

        ## Tevredenheid
        subject = "Tevredenheid"
        sub_attrs = self.page_attrs[subject]
        self.draw_score(c,
                        sub_attrs['score_offset']*cm,
                        sub_attrs['offset_horizontal'],
                        percentage_from_score(sub_attrs['min_score'],sub_attrs['max_score'],getattr(self.person, subject)),
                        sub_attrs['offset_vertical']*cm,
                        height = height)

        c.drawImage("resources/images/arrow_"+self.cutoff(subject)[0]+".png",
                    (-10.93*cm)+(0.4*cm),
                    4.8*cm,
                    height=25,
                    preserveAspectRatio=True)

        frame = Frame(1.5*cm, 2.2*cm,8*cm,100, showBoundary=0)
        story = [Paragraph(self.cutoff(subject)[1], custom_style)]
        frame.addFromList(story,c)

        c.showPage() # Burn-out klachten, bevlogenheid, tevredenheid

        ## Organisatiecultuur
        subject = "Organisatiecultuur"

        sub_attrs = self.page_attrs[subject]
        self.draw_score(c,
                        sub_attrs['score_offset']*cm,
                        sub_attrs['offset_horizontal'],
                        percentage_from_score(sub_attrs['min_score'],sub_attrs['max_score'],getattr(self.person, subject), rotate=True),
                        sub_attrs['offset_vertical']*cm,
                        height = height)

        c.drawImage("resources/images/arrow_"+self.cutoff(subject)[0]+".png",
                    (-10.93*cm)+(0.4*cm),
                    18.7*cm,
                    height=25,
                    preserveAspectRatio=True)

        frame = Frame(1.5*cm, 16.1*cm,8*cm,100, showBoundary=0)
        story = [Paragraph(self.cutoff(subject)[1], custom_style)]
        frame.addFromList(story,c)

        c.showPage() # Organisatiecultuur
        c.showPage()
        c.showPage()
        c.showPage()

        c.save()

        self.save_canvas(self.out_pdf_file,
                         existing_pdf)

In [ ]:
print("Generating PDFs")
print("---")
data = rtrn2['body']
data = json.loads(data)

df = data['dataframe']
variables = data['variables']
class_inf = data['form_info']['class_info']

class_attrs = vitalityIchtus_info['class_attrs'] #class_inf['class_attrs'] # Change this back after it works

out_filename = class_attrs['out_filename']

filepath = general_prefix+"tmp/"+out_filename

if class_inf['class_name'] == 'Vitality':
    _class = Vitality(class_attrs)

elif class_inf['class_name'] == 'Education':
    _class = Education(class_attrs)

elif class_inf['class_name'] == 'Vitality HLR':
    _class = VitalityHLR(class_attrs)

elif class_inf['class_name'] == 'Vitality Ichtus':
    _class = VitalityIchtus(class_attrs)

for key, val in class_attrs.items():
    setattr(_class,key,val)

personList = []

for person in df:
    _person = Person(
        email=person['email']
    )

    for var in variables:
        setattr(_person, var, person[var])

    setattr(_person, "url", person["url"])

    personList.append(_person)

resultsList = []

for person in personList:
    result = {}

    _class.person = person
    _class.init_super()

    # print(_class.__dict__)

    create_canvas = _class.create_canvas()
    create_canvas

    with open(filepath, "rb") as f:
        print("Encoding PDF")
        b = base64.b64encode(f.read()).decode("utf-8")

    result['report'] = b
    result['person'] = json.dumps(person.__dict__)
    result['class_info'] = json.dumps(class_inf)
    result['report_title'] = out_filename
    resultsList.append(result)

pdf_headers = headers.copy()
pdf_headers['Content-Type'] = 'application/pdf'

rtrn3 = {
    'statusCode': 200,
    'headers': pdf_headers,
    'body': json.dumps(resultsList),
    "isBase64Encoded": True
}

In [ ]:
def upload_file_to_s3(file_name, bucket, object_name=None):
    """Upload a file to an S3 bucket

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = os.path.basename(file_name)

    # Upload the file
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(file_name, bucket, object_name, ExtraArgs={'ACL': 'public-read'})
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [ ]:
def send_emails(event, context):
    print("Sending emails")
    print("---")
    data = event['body']
    data = json.loads(data)

    for i, entity in enumerate(data):
        person = json.loads(entity['person'], object_hook=lambda d: SimpleNamespace(**d))
        class_info = json.loads(entity['class_info'])

        filepath = general_prefix + "tmp/" + entity['report_title']

        if person.email != None:
            sender = "Neuro Habits <info@neurohabits.nl>"
            recipient = person.email
            aws_region = "eu-central-1"
            subject = "Uw Quickscan Rapport"

            with open(filepath, 'wb') as f:
                f.write(base64.b64decode(entity['report']))

            print(class_info['class_attrs'])
            send_mail_with_attach_ses(sender,
                                      recipient,
                                      aws_region,
                                      subject,
                                      filepath,
                                      person,
                                      class_info['class_attrs']['template_link'])

        else:
            print("using URL")
            upload_file_to_s3(filepath, "nh-psa", str(person.url)+"/"+entity['report_title'])

        reports_sent = i + 1

    result = {
        'reports_sent': reports_sent
    }

    return {
        'statusCode': 200,
        'headers': headers,
        'body': json.dumps(result)
    }

In [ ]:
send_emails(rtrn3, None)